In [ ]:
# --- Importaciones ---
# Para manejo de datos
import pandas as pd
import re
import string

# Para análisis de sentimiento
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

# Para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# --- Descarga de recursos de NLTK ---
try:
    stopwords.words('spanish')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.download('wordnet')
except Exception as e:
    print(f"Error al descargar 'wordnet': {e}")

try:
    nltk.download('omw-1.4')
except Exception as e:
    print(f"Error al descargar 'omw-1.4': {e}")

try:
    nltk.download('punkt')
except Exception as e:
    print(f"Error al descargar 'punkt': {e}")

try:
    nltk.download('averaged_perceptron_tagger')
except Exception as e:
    print(f"Error al descargar 'averaged_perceptron_tagger': {e}")

try:
    nltk.download('brown')
except Exception as e:
    print(f"Error al descargar 'brown': {e}")

# --- Carga de datos ---
try:
    df = pd.read_csv('./tweets_politica.txt', sep='\t', encoding='latin1')
    print(df.head())
except FileNotFoundError:
    print("Error: El archivo 'tweets_politica.txt' no fue encontrado. Asegúrate de que la ruta sea correcta.")
    exit()

In [ ]:
from textblob import TextBlob

try:
    blob = TextBlob("This is a test.")
    translated = blob.translate(to='es')
    print(f"Traducción exitosa: {translated}")
except Exception as e:
    print(f"Error al probar la traducción: {e}")

In [ ]:
# 1. Cargar los datos
df = pd.read_csv('./tweets_politica.txt', sep='\t', encoding='latin1')

print(df.head())

In [ ]:
# Función de limpieza
def limpiar_texto(texto):
    if isinstance(texto, str):  # Asegurarse de que el texto sea una cadena
        # 1. Pasar a minúsculas
        texto = texto.lower()

        # 2. Eliminar URLs
        texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)

        # 3. Eliminar menciones (@usuario)
        texto = re.sub(r'@\w+', '', texto)

        # 4. Eliminar hashtags (solo el #, pero dejando la palabra si quieres)
        texto = re.sub(r'#', '', texto)

        # 5. Eliminar números
        texto = re.sub(r'\d+', '', texto)

        # 6. Eliminar puntuación
        texto = texto.translate(str.maketrans('', '', string.punctuation))

        # 7. Eliminar caracteres especiales (más amplio)
        texto = re.sub(r'[^\w\s]+', '', texto)

        # 8. Eliminar espacios extra (múltiples espacios por uno solo)
        texto = re.sub(r'\s+', ' ', texto).strip()

        # 9. Eliminar stopwords (palabras vacías)
        stop_words = set(stopwords.words('spanish'))
        palabras = texto.split()
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]
        texto = ' '.join(palabras_filtradas)

        return texto
    return '' # Devolver una cadena vacía si la entrada no es texto

# Aplicar la limpieza de texto
df['tweet_limpio'] = df['tweet'].apply(limpiar_texto)

print("\nDataFrame con tweets limpios:")
print(df.head())

In [ ]:
# Análisis de sentimiento
def obtener_sentimiento(texto):
    analysis = TextBlob(texto)
    # Traducir al inglés para un análisis más preciso (opcional, pero recomendado)
    try:
        analysis_en = analysis.translate(to='en')
        if analysis_en.sentiment.polarity > 0:
            return 'positivo'
        elif analysis_en.sentiment.polarity < 0:
            return 'negativo'
        else:
            return 'neutral'
    except Exception as e:
        print(f"Error al traducir o analizar '{texto}': {e}")
        return 'neutral' # En caso de error, se considera neutral

df['sentimiento'] = df['tweet_limpio'].apply(obtener_sentimiento)

print("\nDataFrame con sentimiento:")
print(df.head())

In [ ]:
# Ver primeros resultados
print(df[['tweet', 'polarity', 'sentimiento']].head())